In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split , KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
    
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)

#==================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
# Step 4: Train SVR Model using selected features
#selected_X = X_scaled[:, np.array(selected_features) == 1]
#=============================================================================================================               
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_RMSE=[]
mean_MMRE=[]
mean_MAE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
print(selected_features)


Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 116079696.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 116076536.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 116074088.0000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 116071760.0000
Epoch 5/5
1/1 [==============================] - 0s 54ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 173739408.0000
Epoch 2/5
7/7 [==============================] - 0s 1000us/step - loss: 173736112.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 173733216.0000
Epoch 4/5
7/7 [==============================] - 0s 999us/step - loss: 173729776.0000
Epoch 5/5
1/1 [==============================] - 0s 36ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 212921584.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 212918128.0000
Epoch 3/5
7/7 [==============

7/7 [==============================] - 0s 1ms/step - loss: 170301904.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 170297840.0000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 170294080.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 116066816.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 116063400.0000
Epoch 3/5
7/7 [==============================] - 0s 999us/step - loss: 116060472.0000
Epoch 4/5
7/7 [==============================] - 0s 903us/step - loss: 116057456.0000
Epoch 5/5
1/1 [==============================] - 0s 38ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 173745456.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 173742272.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 173739600.0000
Epoch 4/5
7/7 [=======================

7/7 [==============================] - 0s 833us/step - loss: 173741824.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 173739472.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 173736592.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 999us/step - loss: 212910976.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 212906352.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 212900928.0000
Epoch 4/5
7/7 [==============================] - 0s 1000us/step - loss: 212895744.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 206154704.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 206152320.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 206150224.0000
Epoch 4/5
7/7 [==================

7/7 [==============================] - 0s 833us/step - loss: 206143760.0000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 206140288.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 170311408.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 170308848.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 170306576.0000
Epoch 4/5
7/7 [==============================] - 0s 1000us/step - loss: 170304704.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 116073776.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 116069984.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 116066856.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 116063880.0000
Epoch 5/5
1/1 [======================

7/7 [==============================] - 0s 833us/step - loss: 116065888.0000
Epoch 4/5
7/7 [==============================] - 0s 999us/step - loss: 116062368.0000
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 173741200.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 173737552.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 173734560.0000
Epoch 4/5
7/7 [==============================] - 0s 666us/step - loss: 173731072.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 212919904.0000
Epoch 2/5
7/7 [==============================] - 0s 1000us/step - loss: 212915248.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 212910032.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 212905008.0000
Epoch 5/5
1/1 [==============

...

7/7 [==============================] - 0s 1000us/step - loss: 170300640.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 116078912.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 116075688.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 116073160.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 116070616.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 997us/step - loss: 173757056.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 173752832.0000
Epoch 3/5
7/7 [==============================] - 0s 666us/step - loss: 173749872.0000
Epoch 4/5
7/7 [==============================] - 0s 834us/step - loss: 173746832.0000
Epoch 5/5
1/1 [==============================] - 0s 36ms/step
Epoch 1/5
7/7 [==============================] - 0s

7/7 [==============================] - 0s 1ms/step - loss: 173740752.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 212915184.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 212910096.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 212904720.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 212899280.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 1000us/step - loss: 206138448.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 206132576.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 206126816.0000
Epoch 4/5
7/7 [==============================] - 0s 832us/step - loss: 206120400.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 999

7/7 [==============================] - 0s 999us/step - loss: 206129072.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 170308080.0000
Epoch 2/5
7/7 [==============================] - 0s 1000us/step - loss: 170304064.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 170300464.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 170297408.0000
Epoch 5/5
1/1 [==============================] - 0s 37ms/step
Epoch 1/5
7/7 [==============================] - 0s 998us/step - loss: 116082824.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 116079400.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 116076576.0000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 116073920.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms

7/7 [==============================] - 0s 2ms/step - loss: 116075864.0000
Epoch 5/5
1/1 [==============================] - 0s 47ms/step
Epoch 1/5
7/7 [==============================] - 0s 2ms/step - loss: 173746464.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 173742848.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 173739984.0000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 173736848.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 212922720.0000
Epoch 2/5
7/7 [==============================] - 0s 666us/step - loss: 212919648.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 212916464.0000
Epoch 4/5
7/7 [==============================] - 0s 834us/step - loss: 212913728.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step -

7/7 [==============================] - 0s 833us/step - loss: 212899376.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 833us/step - loss: 206144272.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 206139472.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 206135232.0000
Epoch 4/5
7/7 [==============================] - 0s 999us/step - loss: 206130704.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 170315696.0000
Epoch 2/5
7/7 [==============================] - 0s 1000us/step - loss: 170311264.0000
Epoch 3/5
7/7 [==============================] - 0s 2ms/step - loss: 170307728.0000
Epoch 4/5
7/7 [==============================] - 0s 2ms/step - loss: 170304832.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Mean Absolute Error mean: 8140.137414368949
Mean Magnitud